In [ ]:
import numpy as np # linear algebra
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt

In [ ]:
train_dataset_path = '/kaggle/input/intel-image-classification/seg_train/seg_train/'
validation_dataset_path = '/kaggle/input/intel-image-classification/seg_test/seg_test/'

In [ ]:
IMG_WIDTH = 150
IMG_HEIGHT = 150
BATCH_SIZE = 32

In [ ]:
train_datagen = ImageDataGenerator(rescale=1.0/255,
                                  zoom_range=0.2,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  fill_mode='nearest')
train_generator = train_datagen.flow_from_directory(train_dataset_path,
                                                   target_size=(IMG_WIDTH, IMG_HEIGHT),
                                                   batch_size=BATCH_SIZE,
                                                   class_mode='categorical',
                                                   shuffle=True)

In [ ]:
validation_datagen = ImageDataGenerator(rescale=1.0/255)
validation_generator = validation_datagen.flow_from_directory(validation_dataset_path,
                                                             target_size=(IMG_WIDTH, IMG_HEIGHT),
                                                             batch_size=BATCH_SIZE,
                                                             class_mode='categorical',
                                                             shuffle=True)

In [ ]:
labels = {value: key for key, value in train_generator.class_indices.items()}

print("Label Mappings for classes present in the training and validation datasets\n")
for key, value in labels.items():
    print(f"{key} : {value}")

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=5, figsize=(15, 12))
idx = 0

for i in range(2):
    for j in range(5):
        label = labels[np.argmax(train_generator[0][1][idx])]
        ax[i, j].set_title(f"{label}")
        ax[i, j].imshow(train_generator[0][0][idx][:, :, :])
        ax[i, j].axis("off")
        idx += 1

plt.tight_layout()
plt.suptitle("Sample Training Images", fontsize=21)
plt.show()

In [ ]:
from keras.layers import Conv2D, Activation, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D, Dense, Dropout
from keras.regularizers import l2
def create_model():
    model = keras.models.Sequential([
        Conv2D(filters=128, kernel_size=(5, 5), padding='valid', input_shape=(150, 150, 3)),
        Activation('relu'),
        MaxPooling2D(pool_size=(2, 2)),
        BatchNormalization(),
        
        Conv2D(filters=64, kernel_size=(3, 3), padding='valid', kernel_regularizer=l2(0.0005)),
        Activation('relu'),
        MaxPooling2D(pool_size=(2, 2)),
        BatchNormalization(),
        
        Conv2D(filters=32, kernel_size=(3, 3), padding='valid', kernel_regularizer=l2(0.0005)),
        Activation('relu'),
        MaxPooling2D(pool_size=(2, 2)),
        BatchNormalization(),
        
        GlobalAveragePooling2D(),
        
        Dense(units=256, activation='relu'),
        Dropout(0.5),
        Dense(units=6, activation='softmax')
    ], name='convnet')
    
    return model

In [ ]:
cnn = create_model()

In [ ]:
cnn.compile(
    loss='categorical_crossentropy',
    optimizer=tf.optimizers.Adam(lr=0.001),
    metrics=['accuracy']    
)
cnn.summary()

In [ ]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5,  restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=np.sqrt(0.1), patience=5)

In [ ]:
history=cnn.fit(
    train_generator,
    epochs=20,
    validation_data=validation_generator,
    callbacks=[es, reduce_lr],
    batch_size=64,
    verbose=1
)

In [ ]:
cnn.history.history.keys()

In [ ]:
f,ax=plt.subplots(2,1,figsize=(10,10)) 

#Assigning the first subplot to graph training loss and validation loss
ax[0].plot(cnn.history.history['loss'],color='b',label='Training Loss')
ax[0].plot(cnn.history.history['val_loss'],color='r',label='Validation Loss')

#Plotting the training accuracy and validation accuracy
ax[1].plot(cnn.history.history['accuracy'],color='b',label='Training  Accuracy')
ax[1].plot(cnn.history.history['val_accuracy'],color='r',label='Validation Accuracy')

plt.legend()

In [ ]:
print('Accuracy Score = ',np.max(history.history['val_accuracy']))

In [ ]:
cnn.save("Alexnet.h5")

In [ ]:
alex = keras.models.load_model("Alexnet.h5")

In [ ]:
from IPython.display import clear_output
!pip install gradio
clear_output()

In [ ]:
import gradio as gr

In [ ]:
labels = ['buildings', 'forest', 'glacier', 'mountains', 'sea', 'street']

In [ ]:
def classify_image(inp):
    img = tf.keras.utils.load_img(
    inp, target_size=(150,150)
    )
    img_array = tf.keras.utils.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0) # Create a batch

    predictions = alex.predict(img_array)[0]
    confidences = {labels[i]: float(predictions[i]) for i in range(6)}
    return confidences


gr.Interface(fn=classify_image, 
             inputs=gr.inputs.Image(type="filepath"), 
             outputs=gr.outputs.Label()).launch(share=True, debug=True, allow_flagging='never')